In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import pickle

In [2]:
# file = open('googleNet_embeddings.pkl', 'rb')
embeddings = np.load('../matrix_factorization/embeddings_alt_arch_with_decoder.npy')
embeddings

array([[-0.0866257 ,  0.00861705, -0.02768877, ...,  0.00064043,
         0.03820464, -0.0443062 ],
       [-0.00733163,  0.00637373, -0.00079712, ...,  0.00984534,
        -0.01724906,  0.04450938],
       [-0.01434905,  0.0084365 , -0.01561192, ...,  0.00348288,
         0.00016413,  0.02360854],
       ...,
       [-0.01827695,  0.00692758, -0.01209303, ...,  0.00339431,
         0.00328189,  0.02521952],
       [ 0.00211651,  0.00280362, -0.0056607 , ...,  0.00286982,
        -0.01519183,  0.05249388],
       [ 0.02291233,  0.00477167,  0.00636004, ...,  0.00610208,
        -0.03793249,  0.08019338]])

In [3]:
train_ids = torch.load("../dataset/train_book_id.pkl")
test_ids = torch.load("../dataset/test_book_id.pkl")
print(train_ids.size())
print(test_ids.size())

torch.Size([4000])
torch.Size([1000])


In [4]:
df = pd.read_csv("../dataset/books_with_genres.csv")
df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,"fantasy, paranormal","mystery, thriller, crime",poetry,romance,non-fiction,children,young-adult,"comics, graphic",category,binary_category
0,NaN,7,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,31.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"fantasy, paranormal",fiction
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",NaN,False,4.26,B000FCKCJC,...,NaN,NaN,NaN,NaN,163.0,NaN,NaN,NaN,non-fiction,non-fiction
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",NaN,False,4.43,B017RORXNI,...,NaN,NaN,NaN,NaN,2.0,109.0,1.0,NaN,children,non-fiction
3,1400041694,44,[],US,en-US,"[{'count': '362', 'name': 'to-read'}, {'count'...",NaN,False,3.75,B002OTKEP6,...,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,non-fiction,non-fiction
4,NaN,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",NaN,False,3.83,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,poetry,non-fiction


In [5]:
X_train = embeddings[0:4000]
X_test = embeddings[4000:]
print(X_train.shape)
print(X_test.shape)

(4000, 50)
(1000, 50)


In [6]:
y_train = df["average_rating"][0:4000]
y_test = df["average_rating"][4000:]
print(y_train.shape)
print(y_test.shape)

(4000,)
(1000,)


In [7]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
svr = SVR()
svr.fit(X_train, y_train)
preds = svr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.17669289568399849
r2 score:  -0.013371815764528838


In [8]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.17370536562711902
r2 score:  0.003762312614947416


In [9]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
preds = regr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.1752648181393863
r2 score:  -0.005181483443390533


In [10]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=10, random_state=0).fit(X_train, y_train)
preds = regr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.191259401
r2 score:  -0.0969138613248639
